# Agent-Based Model of Evans 1997 Migraine CEA using Agents.jl -- Claude V2 

HOME: <https://di4health.github.io>

NOTE: This notebook was created by Antropic's Claude Code. Here was my workflow:

1. I gave Google Gemini and Claude Code the two previous validated analyses that I conducted with R's `rdecision` and Julia's `Decision Programming.jl` and asked them to build an agent-based model (ABM) using Julia's `Agents.jl`. 
2. Compared to Gemini, I found the quality of Claude Code's ABM model better, more concise, and mmore reasoned. Therefore, I asked Claude Code to review Gemini's ABM and to take the best of both models and build and a new and improved ABM model. Claude found areas of improvement and generated the notebook content below. I was impressed.

The notebook below is unedited for your review. Your feedback and questions are welcome. 

---


Tomás Aragón

This notebook replicates the Evans 1997 migraine cost-effectiveness analysis as an agent-based microsimulation using [Agents.jl](https://juliadynamics.github.io/Agents.jl/stable/). Each agent is a migraine patient who walks through the decision tree stochastically, accumulating costs and a utility outcome.

| Approach | Method | Output |
|----------|--------|--------|
| **DecisionProgramming.jl** | Analytical (exact expected values via influence diagram) | Exact E[Cost], E[Utility] |
| **Agents.jl (this notebook)** | Simulation (N patients traverse the tree stochastically) | Monte Carlo estimates ± SE |

Sources:
- Evans, K. W., et al. *PharmacoEconomics* 12(5): 565–77 (1997).
- Briggs, A. H., K. Claxton, and M. Sculpher. *Decision Modelling for Health Economic Evaluation.* Oxford, 2011.

In [1]:
using Agents
using Random
using Statistics
using DataFrames
using Printf

## 1. Agent definition

Each patient records their treatment arm, accumulated cost, utility outcome, and which terminal leaf (A–J) they reached. There are no spatial interactions, so we use `NoSpaceAgent`.

In [2]:
@agent struct Patient(NoSpaceAgent)
    treatment::Symbol          # :sumatriptan or :caffeine
    cost::Float64 = 0.0
    utility::Float64 = 0.0
    leaf::Symbol = :unresolved # terminal node A–J
end

## 2. Decision tree traversal (single-step)

The entire tree is traversed in one step — there are no temporal dynamics, just stochastic branching.

```
Treatment ─┬─ Relief? ── Yes ─┬─ Recurrence? ── No  → A/F (u=1.0)
           │                  └──────────────── Yes → B/G (u=0.9, +drug cost)
           └─ Relief? ── No  ─┬─ Endures?  ─── Yes → C/H (u=−0.3)
                              └─ ED visit  ───┬─ Relief   → D/I (u=0.1)
                                              └─ Hospital → E/J (u=−0.3)
```

In [3]:
function agent_step!(agent, model)
    agent.leaf != :unresolved && return      # already resolved

    p = abmproperties(model)
    rng = abmrng(model)

    # Treatment-specific parameters
    is_suma = agent.treatment == :sumatriptan
    p_relief       = is_suma ? p.p_relief_suma       : p.p_relief_caff
    p_norecurrence = is_suma ? p.p_norecurrence_suma : p.p_norecurrence_caff
    c_drug         = is_suma ? p.c_sumatriptan       : p.c_caffeine

    agent.cost = c_drug                        # initial drug cost

    if rand(rng) < p_relief                    # ── Relief? ──
        if rand(rng) < p_norecurrence          #   No recurrence → A/F
            agent.utility = p.u_relief_norec
            agent.leaf = is_suma ? :A : :F
        else                                   #   Recurrence → B/G (+2nd dose)
            agent.cost += c_drug
            agent.utility = p.u_relief_rec
            agent.leaf = is_suma ? :B : :G
        end
    else                                       # ── No relief ──
        if rand(rng) < p.p_endures             #   Endures → C/H
            agent.utility = p.u_norelief_endures
            agent.leaf = is_suma ? :C : :H
        else                                   #   ED visit
            agent.cost += p.c_ed
            if rand(rng) < p.p_relief_ed       #     ED relief → D/I
                agent.utility = p.u_norelief_ed
                agent.leaf = is_suma ? :D : :I
            else                               #     Hospital → E/J
                agent.cost += p.c_hospital
                agent.utility = p.u_norelief_endures
                agent.leaf = is_suma ? :E : :J
            end
        end
    end
end

agent_step! (generic function with 1 method)

## 3. Model constructor and simulation

Both treatment arms live in a single model. All 14 parameters are stored as model properties (not global constants) so they can be varied for sensitivity analysis.

In [4]:
function evans_abm(; n_per_arm::Int = 1_000_000, seed::Int = 42)
    properties = (
        # Costs ($Can)
        c_sumatriptan = 16.10,
        c_caffeine    = 1.32,
        c_ed          = 63.16,
        c_hospital    = 1093.0,
        # Utilities
        u_relief_norec     = 1.0,
        u_relief_rec       = 0.9,
        u_norelief_endures = -0.30,
        u_norelief_ed      = 0.1,
        # Probabilities
        p_relief_suma       = 0.558,
        p_relief_caff       = 0.379,
        p_norecurrence_suma = 0.594,
        p_norecurrence_caff = 0.703,
        p_endures           = 0.92,
        p_relief_ed         = 0.998,
    )

    model = StandardABM(Patient;
        agent_step!, properties, rng = Random.Xoshiro(seed))

    for _ in 1:n_per_arm
        add_agent!(model; treatment = :sumatriptan)
        add_agent!(model; treatment = :caffeine)
    end
    return model
end

evans_abm (generic function with 1 method)

In [5]:
N = 1_000_000
model = evans_abm(n_per_arm = N, seed = 2026)
step!(model, 1)                               # one step resolves all agents
println("Simulated $(nagents(model)) patients ($(N) per arm)")

Simulated 2000000 patients (1000000 per arm)


## 4. Results

In [6]:
# Collect into DataFrame
df = DataFrame(
    treatment = [a.treatment for a in allagents(model)],
    cost      = [a.cost      for a in allagents(model)],
    utility   = [a.utility   for a in allagents(model)],
    leaf      = [a.leaf      for a in allagents(model)]
)

# Split by arm
d1 = filter(:treatment => ==(:sumatriptan), df)
d2 = filter(:treatment => ==(:caffeine), df)

EC_D1, EU_D1 = mean(d1.cost), mean(d1.utility)
EC_D2, EU_D2 = mean(d2.cost), mean(d2.utility)
ΔC = EC_D1 - EC_D2
ΔU = EU_D1 - EU_D2
ICER = (ΔC / ΔU) * 365                        # annualize 24h → 1 year

println("=== Cost-Effectiveness Analysis (N = $N per arm) ===")
println("Strategy           E[Cost]     E[Utility]")
@printf("Caffeine/Ergot     %8.4f    %8.4f\n", EC_D2, EU_D2)
@printf("Sumatriptan        %8.4f    %8.4f\n", EC_D1, EU_D1)
println()
@printf("Incremental Cost:    %8.4f\n", ΔC)
@printf("Incremental Utility: %8.4f\n", ΔU)
@printf("ICER: \$%.0f Can/QALY\n", ICER)

=== Cost-Effectiveness Analysis (N = 1000000 per arm) ===
Strategy           E[Cost]     E[Utility]
Caffeine/Ergot       4.7207      0.2027
Sumatriptan         22.0557      0.4169

Incremental Cost:     17.3350
Incremental Utility:   0.2142
ICER: $29541 Can/QALY


### Comparison with analytical (exact) reference values

In [7]:
# Analytical reference from DecisionProgramming.jl / R rdecision
ref = (EC_D1 = 22.058057, EU_D1 = 0.4168609,
       EC_D2 = 4.714972,  EU_D2 = 0.2012760, ICER = 29363.0)

println("=== ABM vs Analytical ===")
println("                     ABM          Exact       Diff")
@printf("E[C|Sumatriptan]   %9.4f    %9.4f    %+.4f\n", EC_D1, ref.EC_D1, EC_D1 - ref.EC_D1)
@printf("E[U|Sumatriptan]   %9.4f    %9.4f    %+.6f\n", EU_D1, ref.EU_D1, EU_D1 - ref.EU_D1)
@printf("E[C|Caffeine]      %9.4f    %9.4f    %+.4f\n", EC_D2, ref.EC_D2, EC_D2 - ref.EC_D2)
@printf("E[U|Caffeine]      %9.4f    %9.4f    %+.6f\n", EU_D2, ref.EU_D2, EU_D2 - ref.EU_D2)
@printf("ICER               %9.0f    %9.0f    %+.0f\n",  ICER,  ref.ICER,  ICER - ref.ICER)

=== ABM vs Analytical ===
                     ABM          Exact       Diff
E[C|Sumatriptan]     22.0557      22.0581    -0.0024
E[U|Sumatriptan]      0.4169       0.4169    -0.000006
E[C|Caffeine]         4.7207       4.7150    +0.0057
E[U|Caffeine]         0.2027       0.2013    +0.001393
ICER                   29541        29363    +178


### Path frequencies

In [8]:
# Analytical path probabilities
p_exact = Dict(
    :A => 0.331452, :B => 0.226548, :C => 0.406640, :D => 0.035289, :E => 0.000071,
    :F => 0.266437, :G => 0.112563, :H => 0.571320, :I => 0.049581, :J => 0.000099
)

freq = combine(groupby(df, [:treatment, :leaf]), nrow => :n)
sort!(freq, [:treatment, :leaf])
freq.simulated  = freq.n ./ N
freq.analytical = [p_exact[l] for l in freq.leaf]
freq.diff       = freq.simulated .- freq.analytical
select!(freq, Not(:n))
freq

Row,treatment,leaf,simulated,analytical,diff
,Symbol,Symbol,Float64,Float64,Float64
1,caffeine,F,0.26727,0.266437,0.000833
2,caffeine,G,0.112735,0.112563,0.000172
3,caffeine,H,0.570066,0.57132,-0.001254
4,caffeine,I,0.049839,0.049581,0.000258
5,caffeine,J,9.0e-5,9.9e-5,-9.0e-6
6,sumatriptan,A,0.331155,0.331452,-0.000297
7,sumatriptan,B,0.22697,0.226548,0.000422
8,sumatriptan,C,0.406821,0.40664,0.000181
9,sumatriptan,D,0.034974,0.035289,-0.000315


### Monte Carlo standard errors and 95% confidence intervals

In [9]:
se(x) = std(x) / sqrt(length(x))

println("=== 95% Confidence Intervals ===")
@printf("E[C|Sumatriptan]: %8.4f ± %.4f\n", EC_D1, 1.96 * se(d1.cost))
@printf("E[U|Sumatriptan]: %8.4f ± %.6f\n", EU_D1, 1.96 * se(d1.utility))
@printf("E[C|Caffeine]:    %8.4f ± %.4f\n", EC_D2, 1.96 * se(d2.cost))
@printf("E[U|Caffeine]:    %8.4f ± %.6f\n", EU_D2, 1.96 * se(d2.utility))

=== 95% Confidence Intervals ===
E[C|Sumatriptan]:  22.0557 ± 0.0322
E[U|Sumatriptan]:   0.4169 ± 0.001205
E[C|Caffeine]:      4.7207 ± 0.0344
E[U|Caffeine]:      0.2027 ± 0.001191


## Summary

With 1M agents per arm the ABM converges to the analytical ICER ≈ \$29,363 Can/QALY.

**Advantages of the ABM over the analytical solution:**
- Full **distribution** of individual outcomes (not just expected values)
- **Confidence intervals** directly from Monte Carlo standard errors
- Natural extension to **patient heterogeneity** (age-varying probabilities, comorbidities)
- Straightforward to add **multi-period dynamics** (repeated episodes) or **resource constraints** (ED capacity)

**Trade-off:** DecisionProgramming.jl gives exact answers instantly; the ABM requires many agents for precision but produces richer output.